In [1]:
import logging
logger = logging.getLogger()
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter("=[%(levelname)s @ %(filename)s (L=%(lineno)s) F=%(funcName)s() ]= %(message)s")
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)   

In [2]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
#from generator import ExperimentConfiguration
#from generator import GeneratorConstants

In [3]:
RESULT_TIMING_LOGS = ["nohup_02.txt","nohup_03.txt","nohup_04.txt","nohup_08.txt","nohup_09.txt","nohup_12.txt"]
RESULT_FOLDER = "../results/"

In [4]:
import re

from datetime import datetime

timing_res = {}
errors = []
shapely_warnings = []
experiment_count = 0

In [5]:
def process_line(line):
    global experiment_count
    if line == "":
        #skip empty lines
        return True
    if "%].." in line: # Example: [ID = 11600][3.3.4.3.4][10 %]..
        # Generic progress update line - dismiss
        return None
    if "Starting experiments with config" in line: # Example: =[CRITICAL @ __main__.py (L=34) F=main() ]= Starting experiments with config ``3.3.3.3.3.3`` starting with 1001, for 100 iterations
        # Starting batch of experiments - dismiss
        return None
    if "Executing..." in line: # Example: [ID = 11600][3.3.4.3.4] Executing... @ 12-08-2023, 18:45:27
        experiment_count = experiment_count + 1
        tmp = re.findall(r'\[.*?\]', line)
        id = re.search(r'\d+', tmp[0]).group()
        cfg = tmp[1][1:-1]
        time_match = re.search(r'\d{2}-\d{2}-\d{4}, \d{2}:\d{2}:\d{2}', line)
        timer = datetime.strptime(time_match.group(), '%d-%m-%Y, %H:%M:%S')
        timing_res[id] = {"config": cfg, "start": timer, "analyze": None, "end": None, "total_secs":None}
        #print("EX")
        #Line of experiment starting
        return None
    if "Analyzing.." in line: # Example: [ID = 21401][3.12.12] Analyzing.. @ 07-08-2023, 21:28:24
        #Line of experiment result analysis starting
        tmp = re.findall(r'\[.*?\]', line)
        id = re.search(r'\d+', tmp[0]).group()
        cfg = tmp[1][1:-1]
        time_match = re.search(r'\d{2}-\d{2}-\d{4}, \d{2}:\d{2}:\d{2}', line)
        timer = datetime.strptime(time_match.group(), '%d-%m-%Y, %H:%M:%S')
        if timer is None:
            print("TIMER PARSE ERROR...", line)
        timing_res[id]["analyze"] = timer

        return None
    

    if "DONE @ " in line: # Example: [ID = 11599][3.3.4.3.4] DONE @ 12-08-2023, 18:45:27
        #Finished experiment
        tmp = re.findall(r'\[.*?\]', line)
        id = re.search(r'\d+', tmp[0]).group()
        cfg = tmp[1][1:-1]
        time_match = re.search(r'\d{2}-\d{2}-\d{4}, \d{2}:\d{2}:\d{2}', line)
        timer = datetime.strptime(time_match.group(), '%d-%m-%Y, %H:%M:%S')
        timing_res[id]["end"] = timer
        if timer is None:
            print("TIMER PARSE ERROR...", line)
        timing_res[id]["total_secs"] = (timing_res[id]["end"] - timing_res[id]["start"]).total_seconds()
        
        return None
    if "ERROR - node counts don't match" in line: # Example: =[ERROR @ experiment.py (L=590) F=execute_experiment() ]= [ID = 11599][3.3.4.3.4]ERROR - node counts don't match correctly from two calculations! SHOULD ALLWAYS MATCH... RETRYING!
        tmp = re.findall(r'\[.*?\]', line)
        err = {"id": re.search(r'\d+', tmp[1]).group(), "config": tmp[2][1:-1]}
        errors.append(err)
        return None
    if "RuntimeWarning: invalid value encountered in" in line: #Example: /home/krisjanis.nesenbergs/.local/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
        # Error in Shapely lib....usually leads to wrong result and recalculation
        err = {"id":"shapely", "config": None}
        shapely_warnings.append(err)
        return None
    if "return lib.intersection(a, b, **kwargs)" in line: # Example: return lib.intersection(a, b, **kwargs)
        #Aniother line of shapely warning - no new info = dismiss
        return None
    if "END:" in line: # END:  3.12.12
        # End of experiment group - dismiss
        return None
    return line

In [6]:
iterations = 10000000
timing_res = {}
errors = []
shapely_warnings = []
experiment_count = 0
experiment_offset = 0
import pprint 
pp = pprint.PrettyPrinter(indent=4, depth=None, sort_dicts = False)

for LOG_X in RESULT_TIMING_LOGS:
    experiment_offset = experiment_count
    FULL_PATH = RESULT_FOLDER+LOG_X
    print("Processing Log", LOG_X)
    with open(FULL_PATH) as infile:
        iii=0
        for line in infile:
            if True:
                res = process_line(line)
                if res is not None:
                    print(res)
            iii=iii+1

            if iii>iterations:
                break
        print("Log",LOG_X,"contained", str(experiment_count-experiment_offset), "experiments") 
print("====================")
print("Total experiments processed:", str(experiment_count))

print("ERRORS: ", len(errors))
#pp.pprint(errors)


print("WARNINGS: ", len(shapely_warnings))
#pp.pprint(shapely_warnings)

aggr = {}
for id in timing_res:
    cfg = timing_res[id]["config"]
    if cfg not in aggr:
        aggr[cfg] = {"sum": 0.0, "cnt": 0, "max": 0.0, "min": 99999999.0, "errors": 0 }
    tim = timing_res[id]["total_secs"]
    aggr[cfg]["sum"] = aggr[cfg]["sum"] + tim
    aggr[cfg]["cnt"] = aggr[cfg]["cnt"] + 1
    if aggr[cfg]["max"]<tim:
        aggr[cfg]["max"] = tim
    if aggr[cfg]["min"]>tim:
        aggr[cfg]["min"] = tim

for conf in aggr:
    aggr[conf]["avg"] = aggr[conf]["sum"] / aggr[conf]["cnt"]

err_id = {}  
for er in errors:
    if er["id"] not in err_id:
        err_id[er["id"]] = 0
    err_id[er["id"]] = err_id[er["id"]] + 1     

    aggr[er["config"]]["errors"] = aggr[er["config"]]["errors"] + 1


Processing Log nohup_02.txt
Log nohup_02.txt contained 6500 experiments
Processing Log nohup_03.txt
Log nohup_03.txt contained 5850 experiments
Processing Log nohup_04.txt
Log nohup_04.txt contained 6500 experiments
Processing Log nohup_08.txt
Log nohup_08.txt contained 10400 experiments
Processing Log nohup_09.txt
Log nohup_09.txt contained 5850 experiments
Processing Log nohup_12.txt
Log nohup_12.txt contained 9100 experiments
Total experiments processed: 44200
ERRORS:  2698
WARNINGS:  422


In [8]:
pp.pprint(aggr)
#TODO some sort of additional analysis/graphs for each aggregated config result analysis?

{   '3.3.3.3.3.3': {   'sum': 16731716.0,
                       'cnt': 3400,
                       'max': 54426.0,
                       'min': 168.0,
                       'errors': 491,
                       'avg': 4921.092941176471},
    '4.6.12.b': {   'sum': 8410902.0,
                    'cnt': 3400,
                    'max': 17058.0,
                    'min': 159.0,
                    'errors': 103,
                    'avg': 2473.794705882353},
    '4.6.12.a': {   'sum': 9305675.0,
                    'cnt': 3400,
                    'max': 19119.0,
                    'min': 123.0,
                    'errors': 122,
                    'avg': 2736.9632352941176},
    '4.8.8': {   'sum': 8305863.0,
                 'cnt': 3400,
                 'max': 16702.0,
                 'min': 173.0,
                 'errors': 108,
                 'avg': 2442.900882352941},
    '3.3.4.3.4': {   'sum': 14598770.0,
                     'cnt': 3400,
                     'max': 4199

In [9]:


#pp.pprint(err_cfg)

for eid in err_id:
    #Specific configurations, that got more than 1 error, and thus might be interesting...
    if err_id[eid]>1:
        print("ERR", eid, "=",err_id[eid], "LOG total seconds:", timing_res[eid]["total_secs"])

#pp.pprint(err_id)

ERR 41816 = 2 LOG total seconds: 299.0
ERR 41818 = 2 LOG total seconds: 343.0
ERR 18215 = 2 LOG total seconds: 778.0
ERR 25017 = 2 LOG total seconds: 380.0
ERR 25018 = 2 LOG total seconds: 456.0
ERR 4617 = 4 LOG total seconds: 580.0
ERR 42040 = 2 LOG total seconds: 422.0
ERR 8017 = 6 LOG total seconds: 597.0
ERR 11417 = 2 LOG total seconds: 471.0
ERR 8020 = 3 LOG total seconds: 563.0
ERR 1217 = 2 LOG total seconds: 478.0
ERR 38657 = 2 LOG total seconds: 331.0
ERR 18439 = 2 LOG total seconds: 464.0
ERR 4636 = 2 LOG total seconds: 429.0
ERR 11239 = 2 LOG total seconds: 533.0
ERR 21699 = 2 LOG total seconds: 344.0
ERR 11439 = 2 LOG total seconds: 555.0
ERR 1236 = 3 LOG total seconds: 517.0
ERR 1239 = 2 LOG total seconds: 496.0
ERR 4460 = 2 LOG total seconds: 465.0
ERR 1440 = 2 LOG total seconds: 506.0
ERR 38560 = 2 LOG total seconds: 463.0
ERR 4852 = 2 LOG total seconds: 867.0
ERR 11339 = 2 LOG total seconds: 473.0
ERR 4477 = 2 LOG total seconds: 368.0
ERR 1056 = 2 LOG total seconds: 436.

In [10]:
total_time = 0.0
for ind in aggr:
    total_time = total_time + aggr[ind]['sum']
print(total_time)


145648773.0


In [11]:
print("Total time in seconds:", total_time)

Total time in seconds: 145648773.0


In [12]:
print("Total time in minutes:", total_time/60)

Total time in minutes: 2427479.55


In [13]:
print("Total time in hours:", total_time/3600)

Total time in hours: 40457.9925


In [14]:
print("Total time in days:", (total_time/3600)/24)

Total time in days: 1685.7496875


In [15]:
print("Total time in years:", ((total_time/3600)/24)/365.35)

Total time in years: 4.61406784590119


In [17]:
print("Total time on 32 cores in days:", ((total_time/3600)/24)/32)

Total time on 32 cores in days: 52.679677734375


In [18]:
print("Total time on 6 servers with 32 cores in days:", ((total_time/3600)/24)/(32*6))

Total time on 6 servers with 32 cores in days: 8.779946289062499


In [20]:
#max time
max_time = aggr['3.3.3.3.3.3']['max']

In [22]:
print("Max time for one calculation in hours:", max_time/3600)

Max time for one calculation in hours: 15.118333333333334


In [24]:
#ACTUAL TIME:
#02 x 6500: 07-08-2023, 20:53:54 => 13-08-2023, 08:19:46 = 5 days, 11 hours, 25 minutes, 52 seconds (473152 seconds)
#03 x 5850: 09-08-2023, 06:19:05 => 15-08-2023, 15:28:06 = 6 days, 9 hours, 9 minutes, 1 seconds (551341 seconds)
#04 x 6500: 07-08-2023, 10:28:36 => 15-08-2023, 06:36:11 = 7 days, 20 hours, 7 minutes, 35 seconds (677255 seconds)
#08 x 10400: 05-08-2023, 18:10:22 => 15-08-2023, 10:01:54 = 9 days, 15 hours, 51 minutes, 32 seconds (834692 seconds)
#09 x 5850: 09-08-2023, 06:20:11 => 15-08-2023, 13:17:23 = 6 days, 6 hours, 57 minutes, 12 seconds (543432 seconds)
#12 x 9100: 05-08-2023, 15:09:45 => 13-08-2023, 11:57:37 = 7 days, 20 hours, 47 minutes, 52 seconds (679672 seconds)

# Longest time:
# 05-08-2023, 15:09:45 => 15-08-2023, 15:28:06 = 10 days, 0 hours, 17 minutes, 44 seconds (865064 seconds)
# instead of theoretical 8 days, 18 hours, 43 minutes, 8 seconds
# thus overhead over theoretical due to bad planning = 1 day, 5 hours, 34 minutes, 36 seconds